In [1]:
import os,sqlite3,re,hashlib
import pandas as pd
import numpy as np

In [2]:
# setting
pd.options.display.max_columns = None

In [3]:
# 路径
datapath = os.path.join(r'/home/pi/Documents/data/asoproject/20190723')
dbpath = os.path.join(datapath,'aso20190723.db')
conn = sqlite3.connect(dbpath)
cur  = conn.cursor()

In [18]:
# tool func
def create_table(tablename,var_dict):
    var_define = ','.join(['{} {}'.format(k,v) for k,v in var_dict.items()])
    create_table_cmd = 'create table if not exists {} ({})'.format(tablename,var_define)
    try:
        cur.execute(create_table_cmd)
        conn.commit()
        print('{} created'.format(tablename))
    except:
        print('wrong in create {}'.format(tablename))
def insert_data(tablename, df, datahashornot = True):
    df_columns = df.columns.tolist()
    if datahashornot:
        hash_sha256 = hashlib.sha256()
        insert_cmd = 'insert into {}(datahash,{}) values ({}?)'.format(tablename,','.join(df_columns),'?,'*(len(df_columns)))
    else:
        insert_cmd = 'insert into {}({}) values ({}?)'.format(tablename,','.join(df_columns),'?,'*(len(df_columns)-1))
    for r in df.itertuples(index = False):
        if datahashornot:
            hash_sha256.update(str(tuple(r)).encode())
            datahash = hash_sha256.hexdigest()
            tempdata = (datahash,)+tuple(r)
        else:
            tempdata = tuple(r)
        try:
            cur.execute(insert_cmd,tempdata)
            conn.commit()
        except:
            print('wrong')
    print('complete insert')
def insertlog(logtext):
    tempcmd = 'insert into changelog (date,log) values (?,?)'
    cur.execute(tempcmd,(time.asctime(),logtext))
    conn.commit()

In [5]:
# sql语句
changelog_cmd = 'select * from changelog'

# 读取
changelog = pd.read_sql(changelog_cmd,conn)

print(changelog.loc[0,'log'])

数据来源与字段说明
Table: baseinfo
1. 数据标签：原始数据
2. 数据来源：
数据来源于鸟哥aso网站（aso.niaogebiji.com）网站包含从applestore获得的ios系统的数据。
数据通过爬虫爬取。
2.1 爬虫代码地址：
爬取代码分为两部分
第一部分爬取id来自于七麦数据中2017年每月排名前1000的app
https://github.com/akasiro/asoscrape20190704.git
第二部分爬取根据talkindata给的数据中的包名得到的但不含在第一部分数据中的id
https://github.com/akasiro/asoscrape20190716.git
爬取数据之后将数据合并到数据库中的代码为
https://github.com/akasiro/mergedata-tosqlite.git
3. 数据描述
3.1. 数据简介：
数据来源于鸟哥aso网站（aso.niaogebiji.com）网站包含从applestore获得的ios系统的数据。
数据通过爬虫爬取。
3.2. 样本数量：
13088个appleid * 24个变量
3.3. 变量描述：
appleid（主键）  : 游戏在applestore中的appid
appName  : 游戏名称
appType  : app类型（在样本中的数据均为游戏）
price  : 软件下载的价格
latestVersion  : 爬取数据时的最新版本更新时间
developerFirm  : 开发商
developer  : 开发者
tags  : 在applestore的分类
releaseDate  : 发布日期
lastestDate  : 更新日期
bundleId  : ios系统中的包名
lastestVer  : 在爬取数据时的最新版本号
size  : app安装包大小，单位是mb
payInApp  : 是否有内购
support  : 支持网站，在数据中并没有爬取到这部分
compatibility  : 对于操作系统的兼容性
lang  : 语言
contentRank  : 内容评级
artistname  : 作者（或者公司）名称
samepubappnum  : 同开发者有几个医用
samepubapplist  

# baseinfo

In [6]:
baseinfo_cmd = 'select * from baseinfo'
baseinfo = pd.read_sql(baseinfo_cmd,conn)

print(baseinfo.shape)
print(baseinfo.columns)

(13088, 24)
Index(['appleid', 'appName', 'appType', 'price', 'latestVersion',
       'developerFirm', 'developer', 'tags', 'releaseDate', 'lastestDate',
       'bundleId', 'lastestVer', 'size', 'payInApp', 'support',
       'compatibility', 'lang', 'contentRank', 'artistname', 'samepubappnum',
       'samepubapplist', 'samepubapplistid', 'introcontent', 'rdtimestamp'],
      dtype='object')


In [7]:
baseinfo.head(2)

appleid         appName appType   price latestVersion  \
0  281736535          Enigmo      游戏  ¥12.00    2017-11-17   
1  284480400  Word Warp「已下架」      游戏   ¥6.00    2014-10-27   

           developerFirm              developer             tags  releaseDate  \
0  Pangea Software, Inc.  Pangea Software, Inc.  游戏、智力游戏、娱乐、街机游戏  2008年07月11日   
1                   None           MobilityWare     游戏、智力游戏、文字游戏  2008年07月11日   

   lastestDate                   bundleId lastestVer      size payInApp  \
0  2017年11月17日          com.pangea.Enigmo        6.0  23.06 MB        是   
1  2014年10月27日  com.mobilityware.WordWarp      3.2.2  13.65 MB        否   

  support                                    compatibility lang contentRank  \
0    None  需要 iOS 8.0 或更高版本。与 iPhone、iPad 和 iPod touch 兼容。   英语          4+   
1    None  需要 iOS 6.0 或更高版本。与 iPhone、iPad 和 iPod touch 兼容。   英语          4+   

              artistname  samepubappnum  \
0  Pangea Software, Inc.             23   
1           MobilityWare             31   

                                      samepubapplist  \
0  PangeaVR HD|Air Wings Intergalactic|Air Wings®...   
1  Apple Tree - Hangman For Kids|Blackjack - 赌场纸牌...   

                                    samepubapplistid  \
0  373454824|590972134|496386846|298032129|288713...   
1  287360275|289523017|289265443|288316716|291374...   

                                        introcontent  rdtimestamp  
0  •NETWORKEDMULTIPLAYERMODEWITHGAMECENTER!•Voted...   1215705600  
1  “WordWarpismyfirstgo-togamewhenIfindmyselfouto...   1215705600

In [8]:
select_col = ['appleid','appName','price','tags','rdtimestamp','size','payInApp','compatibility','contentRank','samepubappnum']
baseinfo1 = baseinfo[select_col]

def price_func(x):
    if x == np.nan or type(x) == float:
        return x
    if '免费' in str(x):
        return float(0)
    else:
        return float(re.sub(r'[^\d.]+','',str(x)))
baseinfo1['price'] = baseinfo1['price'].apply(price_func)

def size_func(x):
    if x == np.nan or type(x) == float:
        return x
    else:
        temp = float(re.sub(r'[^\d.]+','',x))
        if temp == 0:
            return np.nan
        else:
            return temp
baseinfo1['size'] = baseinfo1['size'].apply(size_func)


def payinapp_func(x):
    if type(x) == float or type(x) == int or x == np.nan:
        return x
    x = str(x)
    if '是' in x:
        return 1
    elif '否' in x:
        return 0
    else:
        return np.nan
baseinfo1['payInApp'] = baseinfo1['payInApp'].apply(payinapp_func)

def contentrank_func(x):
    if type(x) == int or x == np.nan:
        return x
    if x == None:
        return np.nan
    else:
        temp = int(re.sub(r'[^\d]+','',x))
        if temp == 0:
            return np.nan
        else:
            return temp
baseinfo1['contentRank'] = baseinfo1['contentRank'].apply(contentrank_func)

def compatibility_func(x):
    if type(x) == float or x == np.nan:
        return x
    temp = re.search(r'IOS\s*\d.\d',str(x).upper())
    if temp:
        t =  float(re.search(r'\d.\d',temp.group(0)).group(0))
        return t
    else:
        return np.nan
baseinfo1['compatibility'] = baseinfo1['compatibility'].apply(compatibility_func)

def rdtime_func(x):
    if x == np.nan:
        return x
    else:
        temp = (1514736000 -x)/(60*60*24*365)
        return temp
baseinfo1['age'] = baseinfo1['rdtimestamp'].apply(rdtime_func)

/usr/lib/python3/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/lib/python3/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/lib/python3/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pan

In [9]:
baseinfo1.describe()

appleid         price   rdtimestamp          size      payInApp  \
count  1.308800e+04  13088.000000  1.308800e+04  12611.000000  13088.000000   
mean   1.064744e+09      4.224786  1.458365e+09    222.744876      0.680394   
std    2.354095e+08     24.481734  5.400590e+07    291.510542      0.466342   
min    2.817365e+08      0.000000  1.215706e+09      0.060000      0.000000   
25%    9.990619e+08      0.000000  1.443024e+09     62.890000      0.000000   
50%    1.163673e+09      0.000000  1.480522e+09    130.930000      1.000000   
75%    1.219159e+09      1.000000  1.493395e+09    258.240000      1.000000   
max    1.453470e+09    798.000000  1.559837e+09   2048.000000      1.000000   

       compatibility   contentRank  samepubappnum           age  
count   12389.000000  12611.000000   13088.000000  13088.000000  
mean        6.965155      9.053763      36.761843      1.787500  
std         1.152540      4.574847     117.322480      1.712516  
min         2.200000      4.000000       0.000000     -1.430137  
25%         6.000000      4.000000       1.000000      0.676712  
50%         7.000000      9.000000       5.000000      1.084932  
75%         8.000000     12.000000      26.000000      2.273973  
max         9.300000     17.000000    3253.000000      9.482192

# pkgname_appleid

In [10]:
pkgname_appleid_cmd = 'select * from pkgname_appleid'
pkgname_appleid = pd.read_sql(pkgname_appleid_cmd,conn)
print(pkgname_appleid.shape)
print(pkgname_appleid.columns)

(8031, 5)
Index(['type_code', 'pkgName', 'type', 'qimaiId', 'appleid'], dtype='object')


In [11]:
pkg = pkgname_appleid[['appleid','type_code','type']]

In [12]:
bs_pkg = baseinfo1.merge(pkg, on = 'appleid', how = 'left')
print(bs_pkg.shape)
print(bs_pkg.columns)

(15861, 13)
Index(['appleid', 'appName', 'price', 'tags', 'rdtimestamp', 'size',
       'payInApp', 'compatibility', 'contentRank', 'samepubappnum', 'age',
       'type_code', 'type'],
      dtype='object')


In [13]:
bs_pkg_group = bs_pkg['type_code'].dropna().groupby(bs_pkg['appleid'])
spanning = bs_pkg_group.agg([('span','nunique')])
spanning = spanning.reset_index()

In [14]:
bs_pkg_span = bs_pkg.merge(spanning, on = 'appleid', how = 'left')
print(bs_pkg_span.shape)
print(bs_pkg_span.columns)

(15861, 14)
Index(['appleid', 'appName', 'price', 'tags', 'rdtimestamp', 'size',
       'payInApp', 'compatibility', 'contentRank', 'samepubappnum', 'age',
       'type_code', 'type', 'span'],
      dtype='object')


In [15]:
bs_pkg_span.head()

appleid            appName  price             tags  rdtimestamp   size  \
0  281736535             Enigmo   12.0  游戏、智力游戏、娱乐、街机游戏   1215705600  23.06   
1  284480400     Word Warp「已下架」    6.0     游戏、智力游戏、文字游戏   1215705600  13.65   
2  284752907     Blackjack「已下架」    6.0   游戏、扑克牌游戏、娱乐场游戏   1215705600  51.03   
3  284911691          Dice「已下架」    6.0  游戏、娱乐、桌面游戏、骰子游戏   1215705600    NaN   
4  284943962  Moonlight Mahjong    6.0     游戏、智力游戏、桌面游戏   1215705600  48.20   

   payInApp  compatibility  contentRank  samepubappnum       age type_code  \
0         1            8.0          4.0             23  9.482192   T200200   
1         0            6.0          4.0             31  9.482192       NaN   
2         1            6.0          4.0             31  9.482192       NaN   
3         0            NaN          NaN              0  9.482192       NaN   
4         0            NaN          4.0              7  9.482192       NaN   

   type  span  
0  冒险游戏   1.0  
1   NaN   NaN  
2   NaN   NaN  
3   NaN   NaN  
4   NaN   NaN

# 保存bs_pkg_span到数据库aso20190723.db

In [16]:
# 建表
bs_pkg_span_vardict = {'datahash':'varchar(100) primary key',
                      'appleid':'integer',
                      'appName': 'varchar(50)',
                      'price':'float',
                      'tags':'varchar(50)',
                      'rdtimestamp':'integer',
                      'size':'float',
                      'payInApp':'integer',
                      'compatibility':'float',
                      'contentRank':'integer',
                      'samepubappnum':'integer',
                      'age':'float',
                      'type_code':'varchar(20)',
                      'type':'varchar(20)',
                      'span':'integer'}
create_table('bs_pkg_span20190912',bs_pkg_span_vardict)
insert_data('bs_pkg_span20190912',bs_pkg_span)

bs_pkg_span20190912 created
complete insert


In [17]:
# 校验插入
bi_pkg_span_cmd = 'select * from bs_pkg_span20190912'
test = pd.read_sql(bi_pkg_span_cmd,conn)
print(test.shape)
print(test.columns)
print(test.describe())
print(test.head())

(15861, 15)
Index(['datahash', 'appleid', 'appName', 'price', 'tags', 'rdtimestamp',
       'size', 'payInApp', 'compatibility', 'contentRank', 'samepubappnum',
       'age', 'type_code', 'type', 'span'],
      dtype='object')
            appleid         price   rdtimestamp          size      payInApp  \
count  1.586100e+04  15861.000000  1.586100e+04  15362.000000  15861.000000   
mean   1.055149e+09      3.578400  1.456467e+09    263.461455      0.731858   
std    2.314147e+08     22.339829  5.303448e+07    343.855434      0.443006   
min    2.817365e+08      0.000000  1.215706e+09      0.060000      0.000000   
25%    9.801324e+08      0.000000  1.439136e+09     69.207500      0.000000   
50%    1.142531e+09      0.000000  1.477498e+09    145.450000      1.000000   
75%    1.213766e+09      0.000000  1.492358e+09    303.347500      1.000000   
max    1.453470e+09    798.000000  1.559837e+09   2048.000000      1.000000   

       compatibility   contentRank  samepubappnum           a